# Imports

In [1]:
import pandas as pd
import numpy as np
import pickle
import math
import datetime as dt
import json
import requests
from tqdm import tqdm
import os

In [2]:
import geopandas as gpd
import geojson
import h3
import osm2geojson
import shapely
from shapely import wkt
from shapely.geometry import Point, Polygon, LineString
from geopy.distance import geodesic, great_circle

In [3]:
import plotly.graph_objects as go
import plotly.express as px

In [4]:
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

In [5]:
pd.options.display.max_rows = 300
pd.options.display.max_columns = 300

In [6]:
import warnings
warnings.simplefilter('ignore')

# Data

In [13]:
data_cities_df = pd.read_csv('data_cities.csv', sep=';')

In [14]:
data_cities_df.shape

(5171, 8)

In [15]:
data_cities_df.head()

,Unnamed: 0,type,lon,lat,region,region2,name,population
0,0,node,33.367905,45.190764,Автономна Республіка Крим,n/d,Евпатория,105719.0
1,1,node,34.102486,44.952146,Автономна Республіка Крим,n/d,Симферополь,341799.0
2,2,node,34.409539,44.677112,Автономна Республіка Крим,n/d,Алушта,28919.0
3,3,node,34.389913,45.709376,Автономна Республіка Крим,n/d,Джанкой,36665.0
4,4,node,33.035764,45.308062,Автономна Республіка Крим,n/d,Мирный,4210.0


# OpenRouteService

In [44]:
import openrouteservice as ors

In [17]:
ors_key = '5b3ce3597851110001cf6248f0c57acc954e4ec28233433924ae3ef1'  # Specify your personal API key

In [18]:
client_ors = ors.Client(key=ors_key)

## Route

In [38]:
i, j = 420, 470

In [39]:
data_cities_df.loc[[i,j]]

,Unnamed: 0,type,lon,lat,region,region2,name,population
420,420,node,55.342201,55.107124,Башкортостан,Башкортостан,Кушнаренково,9251.0
470,470,way,55.903379,53.381139,Башкортостан,Башкортостан,Салават,NaN


In [73]:
coords = [
    [
        data_cities_df.loc[i,'lon'],
        data_cities_df.loc[i,'lat']
    ],
    [
        data_cities_df.loc[j,'lon'],
        data_cities_df.loc[j,'lat']
    ]
]
coords

[[55.342201, 55.107124], [55.9033788, 53.3811386]]

In [100]:
%%time
routes_ors = client_ors.directions(coords)

CPU times: user 5.72 ms, sys: 282 µs, total: 6.01 ms
Wall time: 107 ms


In [101]:
type(routes_ors)

dict

In [102]:
routes

{'routes': [{'summary': {'distance': 230375.7, 'duration': 11350.7},
   'segments': [{'distance': 230375.7,
     'duration': 11350.7,
     'steps': [{'distance': 75.1,
       'duration': 18.0,
       'type': 11,
       'instruction': 'Head west on Советская улица',
       'name': 'Советская улица',
       'way_points': [0, 4]},
      {'distance': 99.2,
       'duration': 23.8,
       'type': 1,
       'instruction': 'Turn right onto улица Кутуева',
       'name': 'улица Кутуева',
       'way_points': [4, 5]},
      {'distance': 236.8,
       'duration': 28.4,
       'type': 0,
       'instruction': 'Turn left onto 3-й переулок Красноармейской',
       'name': '3-й переулок Красноармейской',
       'way_points': [5, 6]},
      {'distance': 916.3,
       'duration': 66.0,
       'type': 0,
       'instruction': 'Turn left onto Красноармейская улица',
       'name': 'Красноармейская улица',
       'way_points': [6, 17]},
      {'distance': 8129.9,
       'duration': 516.0,
       'type': 

In [103]:
len(routes_ors['routes'])

1

In [104]:
route_ors = routes_ors['routes'][0]['geometry']

In [105]:
route_gj = ors.convert.decode_polyline(route_ors)

In [106]:
type(route_gj)

dict

In [107]:
route_gj

{'type': 'LineString',
 'coordinates': [[55.34237, 55.10752],
  [55.34237, 55.10752],
  [55.34213, 55.10761],
  [55.34182, 55.10781],
  [55.34159, 55.10801],
  [55.34285, 55.10854],
  [55.34076, 55.1103],
  [55.33918, 55.10959],
  [55.33841, 55.10928],
  [55.33679, 55.10862],
  [55.33677, 55.10861],
  [55.33567, 55.10816],
  [55.33539, 55.10805],
  [55.33331, 55.10724],
  [55.33095, 55.10632],
  [55.3303, 55.10606],
  [55.33009, 55.10599],
  [55.32897, 55.10556],
  [55.33104, 55.10334],
  [55.33161, 55.10274],
  [55.33177, 55.10256],
  [55.33185, 55.10248],
  [55.33245, 55.10183],
  [55.33292, 55.10137],
  [55.33331, 55.10099],
  [55.335, 55.09918],
  [55.33506, 55.09912],
  [55.33687, 55.09719],
  [55.33815, 55.09576],
  [55.34045, 55.09359],
  [55.34241, 55.09205],
  [55.34452, 55.09073],
  [55.34673, 55.08959],
  [55.36393, 55.08225],
  [55.36526, 55.08175],
  [55.36659, 55.08135],
  [55.36813, 55.08089],
  [55.36978, 55.08044],
  [55.37295, 55.07982],
  [55.37325, 55.07975],
  [55.

In [118]:
map_route = folium.Map(location=[54, 55], zoom_start=6, width=900, height=550, control_scale=True)

folium.GeoJson(route_gj).add_to(map_route)

map_route

## Matrix

In [109]:
i, j

(420, 470)

In [110]:
coords_2 = [
    [
        data_cities_df.loc[k,'lon'],
        data_cities_df.loc[k,'lat']
    ]
    for k in range(i, j)
]

In [111]:
len(coords_2)

50

In [112]:
coords_2[:3]

[[55.342201, 55.107124], [54.931702, 56.27076], [56.168819, 54.367451]]

In [115]:
[list(reversed(c)) for c in coords_2[:3]]

[[55.107124, 55.342201], [56.27076, 54.931702], [54.367451, 56.168819]]

In [120]:
map_matrix = folium.Map(location=[54, 55], zoom_start=6, width=900, height=550, control_scale=True)

for c in coords_2:
    folium.CircleMarker(
        location=list(reversed(c)),
        radius = 3,
        fill_color='red',
        color=None,
        fill_opacity = 1,
        popup=None
    ).add_to(map_matrix)

map_matrix

In [122]:
%%time
matrix_ors = client_ors.distance_matrix(
    locations=coords_2,
    # profile='foot-walking',
    metrics=['distance'],
    validate=False,
)

CPU times: user 7.12 ms, sys: 0 ns, total: 7.12 ms
Wall time: 846 ms


In [124]:
type(matrix_ors)

dict

In [123]:
matrix_ors

{'distances': [[0.0,
   172757.31,
   123372.2,
   157074.11,
   307627.38,
   321008.59,
   71610.41,
   313073.16,
   475355.25,
   488315.53,
   422406.91,
   153128.09,
   205773.13,
   234199.52,
   150035.47,
   143692.42,
   112681.77,
   89631.71,
   50545.39,
   111890.48,
   116749.63,
   139798.28,
   121365.63,
   142704.56,
   227250.33,
   215685.66,
   209592.48,
   361331.03,
   353124.59,
   313156.31,
   148265.84,
   254583.44,
   123572.34,
   396192.34,
   96610.54,
   170479.91,
   114598.71,
   408641.63,
   156728.2,
   168073.36,
   250876.53,
   152305.95,
   81098.38,
   307650.5,
   200165.34,
   186309.0,
   233164.31,
   496424.34,
   280449.28,
   81567.88],
  [172903.23,
   0.0,
   265612.69,
   299314.59,
   449867.84,
   463249.09,
   241301.72,
   455313.66,
   617595.75,
   630556.0,
   564647.44,
   295368.56,
   348013.59,
   376440.0,
   38880.91,
   42091.07,
   61815.7,
   174169.31,
   178009.0,
   172595.84,
   241790.72,
   264839.38,
   2910

In [125]:
len(matrix_ors['distances'])

50

In [126]:
len(matrix_ors['distances'][0])

50

In [127]:
matrix_ors['distances'][0]

[0.0,
 172757.31,
 123372.2,
 157074.11,
 307627.38,
 321008.59,
 71610.41,
 313073.16,
 475355.25,
 488315.53,
 422406.91,
 153128.09,
 205773.13,
 234199.52,
 150035.47,
 143692.42,
 112681.77,
 89631.71,
 50545.39,
 111890.48,
 116749.63,
 139798.28,
 121365.63,
 142704.56,
 227250.33,
 215685.66,
 209592.48,
 361331.03,
 353124.59,
 313156.31,
 148265.84,
 254583.44,
 123572.34,
 396192.34,
 96610.54,
 170479.91,
 114598.71,
 408641.63,
 156728.2,
 168073.36,
 250876.53,
 152305.95,
 81098.38,
 307650.5,
 200165.34,
 186309.0,
 233164.31,
 496424.34,
 280449.28,
 81567.88]

In [172]:
arr = np.array([1,2,3])
arr

array([1, 2, 3])

In [173]:
print(*arr)

1 2 3


In [174]:
arr.dtype

dtype('int64')

In [175]:
arrs = np.array([1,2,3]), np.array([4,5,6])
arrs

(array([1, 2, 3]), array([4, 5, 6]))

In [176]:
print(*arrs)

[1 2 3] [4 5 6]


In [177]:
np.ix_(*arrs)

(array([[1],
        [2],
        [3]]),
 array([[4, 5, 6]]))

In [178]:
for i, a in enumerate(np.ix_(*arrs)):
    print(i)
    print(a)

0
[[1]
 [2]
 [3]]
1
[[4 5 6]]


In [179]:
[len(a) for a in arrs]

[3, 3]

In [180]:
[len(arrs)]

[2]

In [181]:
[len(a) for a in arrs] + [len(arrs)]

[3, 3, 2]

In [211]:
np.empty([len(a) for a in arrs] + [len(arrs)])

array([[[4.67607173e-310, 0.00000000e+000],
        [0.00000000e+000, 0.00000000e+000],
        [0.00000000e+000, 0.00000000e+000]],

       [[0.00000000e+000, 0.00000000e+000],
        [0.00000000e+000, 0.00000000e+000],
        [0.00000000e+000, 0.00000000e+000]],

       [[0.00000000e+000, 0.00000000e+000],
        [0.00000000e+000, 0.00000000e+000],
        [0.00000000e+000, 0.00000000e+000]]])

In [192]:
la = len(arrs)
arr = np.empty([len(a) for a in arrs] + [la], dtype='int64')
for i, a in enumerate(np.ix_(*arrs)):
    print(i)
    print(a)
    arr[...,i] = a

0
[[1]
 [2]
 [3]]
1
[[4 5 6]]


In [191]:
arr

array([[[1, 4],
        [1, 5],
        [1, 6]],

       [[2, 4],
        [2, 5],
        [2, 6]],

       [[3, 4],
        [3, 5],
        [3, 6]]])

In [187]:
def f_cartesian_product(arrays):
    la = len(arrays)
    dtype = np.find_common_type([a.dtype for a in arrays], [])
    arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
    for i, a in enumerate(np.ix_(*arrays)):
        arr[...,i] = a
    return arr.reshape(-1, la)

In [188]:
f_cartesian_product((np.array([1,2,3]), np.array([4,5,6])))

array([[1, 4],
       [1, 5],
       [1, 6],
       [2, 4],
       [2, 5],
       [2, 6],
       [3, 4],
       [3, 5],
       [3, 6]])

In [243]:
x = np.array([1,2,3])
y = np.array([4,5,6])
np.tile(x, len(y)), np.repeat(y, len(x))

(array([1, 2, 3, 1, 2, 3, 1, 2, 3]), array([4, 4, 4, 5, 5, 5, 6, 6, 6]))

In [237]:
x = np.array([1,2,3])
y = np.array([4,5,6])
np.transpose([np.tile(x, len(y)), np.repeat(y, len(x))])

array([[1, 4],
       [2, 4],
       [3, 4],
       [1, 5],
       [2, 5],
       [3, 5],
       [1, 6],
       [2, 6],
       [3, 6]])

In [244]:
x = np.array([1,2,3])
y = np.array([4,5,6])
np.meshgrid(x, y)

[array([[1, 2, 3],
        [1, 2, 3],
        [1, 2, 3]]),
 array([[4, 4, 4],
        [5, 5, 5],
        [6, 6, 6]])]

In [246]:
x = np.array([1,2,3])
y = np.array([4,5,6])
np.dstack(np.meshgrid(x, y))

array([[[1, 4],
        [2, 4],
        [3, 4]],

       [[1, 5],
        [2, 5],
        [3, 5]],

       [[1, 6],
        [2, 6],
        [3, 6]]])

In [245]:
x = np.array([1,2,3])
y = np.array([4,5,6])
np.dstack(np.meshgrid(x, y)).reshape(-1, 2)

array([[1, 4],
       [2, 4],
       [3, 4],
       [1, 5],
       [2, 5],
       [3, 5],
       [1, 6],
       [2, 6],
       [3, 6]])

In [200]:
f_cartesian_product((np.array([[1,2],[3,4]]), np.array([[5,6],[7,8]])))

ValueError: Cross index must be 1 dimensional

In [255]:
x = np.array([[1,2],[3,4]])
y = np.array([[5,6],[7,8]])
np.tile(x, len(y)), np.repeat(y, len(x))

(array([[1, 2, 1, 2],
        [3, 4, 3, 4]]),
 array([5, 5, 6, 6, 7, 7, 8, 8]))

In [256]:
np.repeat(y, 2, axis=0)

array([[5, 6],
       [5, 6],
       [7, 8],
       [7, 8]])

In [257]:
np.repeat(y, 2, axis=1)

array([[5, 5, 6, 6],
       [7, 7, 8, 8]])

In [260]:
np.vstack([y]*3)

array([[5, 6],
       [7, 8],
       [5, 6],
       [7, 8],
       [5, 6],
       [7, 8]])

In [253]:
np.repeat(y, [2, 2], axis=0)

array([[5, 6],
       [5, 6],
       [7, 8],
       [7, 8]])

In [252]:
np.repeat(y, [2, 2], axis=1)

array([[5, 5, 6, 6],
       [7, 7, 8, 8]])

In [254]:
x = np.array([[1,2],[3,4]])
y = np.array([[5,6],[7,8]])
np.meshgrid(x, y)

[array([[1, 2, 3, 4],
        [1, 2, 3, 4],
        [1, 2, 3, 4],
        [1, 2, 3, 4]]),
 array([[5, 5, 5, 5],
        [6, 6, 6, 6],
        [7, 7, 7, 7],
        [8, 8, 8, 8]])]

In [262]:
x = np.array([[1,2],[3,4]])
y = np.array([[5,6],[7,8]])
np.tile(x, len(y)), np.vstack([y] * len(x))

(array([[1, 2, 1, 2],
        [3, 4, 3, 4]]),
 array([[5, 6],
        [7, 8],
        [5, 6],
        [7, 8]]))

In [265]:
x = np.array([[1,2],[3,4]])
y = np.array([[5,6],[7,8]])
np.tile(x, len(y)).ravel(), np.vstack([y] * len(x)).ravel()

(array([1, 2, 1, 2, 3, 4, 3, 4]), array([5, 6, 7, 8, 5, 6, 7, 8]))

In [267]:
x = np.array([[1,2],[3,4]])
y = np.array([[5,6],[7,8]])
np.dstack([np.tile(x, len(y)).ravel(), np.vstack([y] * len(x)).ravel()])

array([[[1, 5],
        [2, 6],
        [1, 7],
        [2, 8],
        [3, 5],
        [4, 6],
        [3, 7],
        [4, 8]]])

In [268]:
x = np.array([[1,2],[3,4]])
y = np.array([[5,6],[7,8]])
np.dstack([np.tile(x, len(y)).ravel(), np.vstack([y] * len(x)).ravel()]).reshape(-1,4)

array([[1, 5, 2, 6],
       [1, 7, 2, 8],
       [3, 5, 4, 6],
       [3, 7, 4, 8]])

In [206]:
for i, a1 in enumerate(np.array([[1,2],[3,4]])):
    for j, a2 in enumerate(np.array([[5,6],[7,8]])):
        print(i*2 + j)
        print(np.array([a1, a2]))

0
[[1 2]
 [5 6]]
1
[[1 2]
 [7 8]]
2
[[3 4]
 [5 6]]
3
[[3 4]
 [7 8]]


In [216]:
np_empty = np.empty([4,2,2], dtype='float64')
np_empty

array([[[4.67607093e-310, 0.00000000e+000],
        [            nan,             nan]],

       [[            nan,             nan],
        [            nan,             nan]],

       [[            nan,             nan],
        [            nan,             nan]],

       [[            nan,             nan],
        [            nan,             nan]]])

In [217]:
np_empty[0]

array([[4.67607093e-310, 0.00000000e+000],
       [            nan,             nan]])

In [218]:
def f_cartesian_product_2(arr1, arr2):
    la1 = len(arr1)
    la2 = len(arr2)
    arr = np.empty([la1 * la2, 2, 2], dtype='float64')
    for i, a1 in enumerate(arr1):
        for j, a2 in enumerate(arr2):
            arr[i*la2 + j] = np.array([a1, a2])
    return arr

In [219]:
f_cartesian_product_2(np.array([[1,2],[3,4]]), np.array([[5,6],[7,8]]))

array([[[1., 2.],
        [5., 6.]],

       [[1., 2.],
        [7., 8.]],

       [[3., 4.],
        [5., 6.]],

       [[3., 4.],
        [7., 8.]]])

In [220]:
f_cartesian_product_2(np.array([[1,2],[3,4]]), np.array([[5,6],[7,8]]))[:,0]

array([[1., 2.],
       [1., 2.],
       [3., 4.],
       [3., 4.]])

In [221]:
f_cartesian_product_2(np.array([[1,2],[3,4]]), np.array([[5,6],[7,8]]))[...,0]

array([[1., 5.],
       [1., 7.],
       [3., 5.],
       [3., 7.]])

In [225]:
f_cartesian_product_2(np.array([[1,2],[3,4]]), np.array([[5,6],[7,8]]))[:,:,0]

array([[1., 5.],
       [1., 7.],
       [3., 5.],
       [3., 7.]])

In [226]:
f_cartesian_product_2(np.array([[1,2],[3,4]]), np.array([[5,6],[7,8]]))[:,:,0][:,0]

array([1., 1., 3., 3.])

In [228]:
f_cartesian_product_2(np.array([[1,2],[3,4]]), np.array([[5,6],[7,8]]))[:,:,0][:,1]

array([5., 7., 5., 7.])

In [229]:
f_cartesian_product_2(np.array([[1,2],[3,4]]), np.array([[5,6],[7,8]]))[:,:,1][:,0]

array([2., 2., 4., 4.])

In [285]:
data_cities_df[['lon', 'lat']]

,lon,lat
0,33.367905,45.190764
1,34.102486,44.952146
2,34.409539,44.677112
3,34.389913,45.709376
4,33.035764,45.308062
...,...,...
5166,39.541914,57.867485
5167,40.686662,58.357637
5168,39.121489,58.506269
5169,40.171550,58.185976


In [284]:
np.dstack([data_cities_df['lon'].values, data_cities_df['lat'].values])

array([[[33.3679049, 45.1907635],
        [34.1024858, 44.9521457],
        [34.4095393, 44.677112 ],
        ...,
        [39.1214888, 58.5062693],
        [40.17155  , 58.1859764],
        [38.456599 , 57.7850575]]])

In [288]:
coords_full = np.dstack([data_cities_df['lon'].values, data_cities_df['lat'].values])[0]

In [289]:
len(coords_full)

5171

In [290]:
x = coords_full
y = coords_full

In [291]:
%%time
cp1 = np.dstack([np.tile(x, len(y)).ravel(), np.vstack([y] * len(x)).ravel()]).reshape(-1,4)

CPU times: user 232 ms, sys: 255 ms, total: 487 ms
Wall time: 491 ms


In [314]:
cp1[:10]

array([[33.3679049, 33.3679049, 45.1907635, 45.1907635],
       [33.3679049, 34.1024858, 45.1907635, 44.9521457],
       [33.3679049, 34.4095393, 45.1907635, 44.677112 ],
       [33.3679049, 34.3899131, 45.1907635, 45.7093755],
       [33.3679049, 33.0357639, 45.1907635, 45.3080619],
       [33.3679049, 33.8534782, 45.1907635, 44.7522963],
       [33.3679049, 35.2381272, 45.1907635, 44.9633478],
       [33.3679049, 35.8222792, 45.1907635, 45.4291446],
       [33.3679049, 35.7758985, 45.1907635, 45.297284 ],
       [33.3679049, 36.2919984, 45.1907635, 45.3761402]])

In [292]:
%%time
cp2 = f_cartesian_product_2(x, y)

CPU times: user 33.1 s, sys: 0 ns, total: 33.1 s
Wall time: 33.1 s


In [315]:
cp2[:10]

array([[[33.3679049, 45.1907635],
        [33.3679049, 45.1907635]],

       [[33.3679049, 45.1907635],
        [34.1024858, 44.9521457]],

       [[33.3679049, 45.1907635],
        [34.4095393, 44.677112 ]],

       [[33.3679049, 45.1907635],
        [34.3899131, 45.7093755]],

       [[33.3679049, 45.1907635],
        [33.0357639, 45.3080619]],

       [[33.3679049, 45.1907635],
        [33.8534782, 44.7522963]],

       [[33.3679049, 45.1907635],
        [35.2381272, 44.9633478]],

       [[33.3679049, 45.1907635],
        [35.8222792, 45.4291446]],

       [[33.3679049, 45.1907635],
        [35.7758985, 45.297284 ]],

       [[33.3679049, 45.1907635],
        [36.2919984, 45.3761402]]])

In [293]:
def f_cartesian_product_3(arr1, arr2):
    la1 = len(arr1)
    la2 = len(arr2)
    arr = np.dstack([np.tile(arr1, la2).ravel(), np.vstack([arr2] * la1).ravel()]).reshape(-1,4)
    return arr

In [294]:
f_cartesian_product_3(np.array([[1,2],[3,4]]), np.array([[5,6],[7,8]]))

array([[1, 5, 2, 6],
       [1, 7, 2, 8],
       [3, 5, 4, 6],
       [3, 7, 4, 8]])

In [295]:
f_cartesian_product_3(np.array([[1,2],[3,4]]), np.array([[5,6],[7,8]]))[:,0]

array([1, 1, 3, 3])

In [300]:
def f_dist_points_to_points(points_1, points_2):
    
    points_to_points = f_cartesian_product_3(points_1, points_2)
    
    lon_1 = math.pi/180 * points_to_points[:,0]
    lon_2 = math.pi/180 * points_to_points[:,1]
    lat_1 = math.pi/180 * points_to_points[:,2]
    lat_2 = math.pi/180 * points_to_points[:,3]
    
    dlon = lon_1 - lon_2
    dlat = lat_1 - lat_2
    a = np.sin(dlat/2)**2 + np.cos(lat_1) * np.cos(lat_2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    dists = c * 6371.009
    
    return points_to_points, dists

In [301]:
coords

[[55.342201, 55.107124], [55.9033788, 53.3811386]]

In [302]:
f_dist_points_to_points(np.array([coords[0]]), np.array([coords[1]]))

(array([[55.342201 , 55.9033788, 55.107124 , 53.3811386]]),
 array([195.35232034]))

In [312]:
matrix_ors['distances'][0][:10]

[0.0,
 172757.31,
 123372.2,
 157074.11,
 307627.38,
 321008.59,
 71610.41,
 313073.16,
 475355.25,
 488315.53]

In [304]:
len(matrix_ors['distances'])

50

In [305]:
len(matrix_ors['distances'][0])

50

In [307]:
%%time
points_to_points, dists = f_dist_points_to_points(coords_2, coords_2)

CPU times: user 2.08 ms, sys: 0 ns, total: 2.08 ms
Wall time: 1.88 ms


In [308]:
len(dists)

2500

In [313]:
list(dists[:10])

[0.0,
 131.92336265489018,
 97.87936014555544,
 121.00408106408726,
 194.3649965686634,
 234.93390789949044,
 57.60428425629405,
 269.23633146937743,
 340.04611360163506,
 348.1154612051305]